# Basic segmentation of bacteria colonies

**Author:** Daniela Azucena García Soriano     
**Email:** daniela.garcia@chalmers.se  
**Date:** November 10, 2025  
**Institution:** Chalmers University

The goal of this tutorial is the automated identification of bacterial colonies to extract key morphological data and generate labels that ensure tracking of temporal development using basic thresholding. The example data comes from pictures of petri dish plates where soil bacteria has been growing at room temperature for at least 10 days.

![example](output.png)


In [26]:
from google.colab import userdata

In [ ]:
!pip install omero-py https://github.com/glencoesoftware/zeroc-ice-py-linux-x86_64/releases/download/20240202/zeroc_ice-3.6.5-cp311-cp311-manylinux_2_28_x86_64.whl

### Calling required libraries

In this notebook, we are utilizing several powerful Python libraries for data manipulation, image processing, scientific computing, and machine learning. Below is an introduction to each of these libraries and their key functionalities:

1. **omero-py** is a Python client library for OMERO (Open Microscopy Environment Remote Objects), a platform for managing, visualizing, and analyzing large-scale microscopy image datasets. omero-py allows users to interact with an OMERO server programmatically, enabling image uploads, metadata management, and advanced data analysis workflows.
2. **NumPy** is a fundamental package for scientific computing in Python. It provides support for large multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.
3. **Matplotlib** is a plotting library used for creating static, interactive, and animated visualizations in Python. It is particularly useful for generating plots, histograms, bar charts, and other types of visualizations.
4. **Pandas** is a powerful data manipulation and analysis library. It provides data structures like DataFrames, which are essential for handling and analyzing structured data.
5. **SciPy** is a library used for scientific and technical computing. It builds on NumPy and provides a large number of higher-level functions that are useful for different types of scientific and engineering applications, such as optimization, integration, interpolation, and image processing.
6. **OpenCV (cv2)** is an open-source computer vision and image processing library. It provides a wide range of functions for image and video processing, including reading and writing images, filtering, edge detection, and object detection.
7. **scikit-image (skimage)** is a collection of algorithms for image processing. It is built on top of NumPy and is designed to work with other scientific computing libraries. It provides functions for image filtering, morphology, segmentation, and feature extraction.
8. **scikit-learn (sklearn)** is a machine learning library that provides simple and efficient tools for data mining and data analysis. It includes a wide range of algorithms for classification, regression, clustering, and dimensionality reduction, as well as tools for model selection and evaluation.


These libraries together provide a comprehensive toolkit for handling and analyzing data, processing images, and applying machine learning techniques.

In [9]:
%matplotlib inline
from omero.gateway import BlitzGateway
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import ndimage as ndi

import cv2

# presprocessing
import skimage.filters
from skimage.exposure import rescale_intensity

#morphology
from skimage import measure
import skimage.morphology
from skimage.measure import label, regionprops_table

# import threshold and gray convertor functions
from skimage.color import rgb2gray, label2rgb

# for watershed implementation
from skimage.segmentation import watershed
from skimage.feature import peak_local_max

from sklearn.cluster import KMeans


In [10]:
# function to show image
def show_image(image, cmap_type='gray'):
    plt.imshow(image, cmap=cmap_type)
    plt.axis('off')
    plt.show()

### 1. Fetching data from OMERO server
First we will fetch the data from the OMERO server.

In [ ]:
conn = BlitzGateway('test-user', userdata.get('pass'),
                        host='omero.scilifelab.se', secure=True)
conn.connect()

dataset_id = 301
dataset = conn.getObject("Dataset", dataset_id)
images = []
for image in dataset.listChildren():
  images.append(image)
  if len(images) == 0:
    print(None)

for image in images:
  print("---- Image", image.id)

In [ ]:
image_id = 351
image_obj = conn.getObject("Image", image_id)

# Get the raw pixel data
pixels = image_obj.getPrimaryPixels()
channels = []
for c in range(image_obj.getSizeC()):
    plane = pixels.getPlane(0, c, 0)  # z, c, t indices
    channels.append(plane)

# stack the channels to create a color image
image_color = np.stack(channels, axis=-1)
# rescale the intensity of the image to the range [0, 255]
image_color = rescale_intensity(image_color, in_range='image', out_range=(0, 255)).astype(np.uint8)

# get grayscale image
image_gray = pixels.getPlane(0, 0, 0)  # z, c, t indices

# Display the color image using matplotlib
show_image(image_color)

conn.close()

Next, we convert the RGB image to grayscale and adjust the intensity by removing the percentile 10th and 90th (extreme values) from the range.

In [ ]:
p10, p90 = np.percentile(image_gray, (10,90))
image_rescaled = rescale_intensity(image_gray, in_range=(p10, p90))
# rescale the intensity of the image to the range [0, 1]
image_rescaled = image_rescaled / 65535.0
show_image(image_rescaled)

#### 2. Mask and segmentation
We are going to create a mask to remove the area outside of the plate. We define a center coordinates and radius and apply it to the grayscale image.

In [ ]:
imageSize = image_rescaled.shape
ci = [1030, 1050, 840]
# generate grid same size as the original image
x = np.arange(0,imageSize[0])-ci[0]
y = np.arange(0,imageSize[1])-ci[1]
xx, yy = np.meshgrid(x, y)
# generate mask
mask = (xx**2 + yy**2) < ci[2]**2
# turn mask into integer
mask = mask.astype(int)
rgbI_mask=mask*image_rescaled
show_image(rgbI_mask)

To be able to segment our bacterial colonies, we plot the intensity values. We will use to different thresholds to be able to segment bacteria colonies with a bright color and bacteria colonies with a dark color.

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(rgbI_mask.ravel(), bins=100, color='black', alpha=0.7)
plt.xlabel('Intensity Value')
plt.ylabel('Frequency')
plt.title('Histogram of image intensities in grayscale')
plt.show()

From the mask grayscale image, we identified the bright colonies by applying a thresholding at the arbitrary value of > than 0.8 and performing an area closing operation to fill small holes within the colonies.

In [ ]:
image_threshold_bright = rgbI_mask >= 0.80
image_area_closing = skimage.morphology.area_closing(image_threshold_bright, area_threshold=100)
binary_image_bright = image_area_closing
show_image(binary_image_bright)

Next, we identified the dark colonies by applying a thresholding at between the values 0.1 and 0.25. We again do an area closing operation to fill small holes within the colonies.

In [ ]:
image_threshold_dark = (rgbI_mask > 0) & (rgbI_mask < 0.25)
image_area_closing = skimage.morphology.area_closing(image_threshold_dark, area_threshold=100)
binary_image_dark = image_area_closing
show_image(binary_image_dark)

We combine both thresholding outputs to collect bright and dark colonies in a single binary image.

In [ ]:
binary_image = (binary_image_bright + binary_image_dark) * mask
show_image(binary_image)

#### 3. Extract information from regions
To extract the information from each segmented object we use the label() function and the we analyse the properties in the labelled regions. The function labels connected components in the binary image, assigning a unique label to each connected region. The resulting image_labeled is an image where each pixel value correspondos to the label of the region it belongs.

In [ ]:
# label image for mapping
image_labeled = label(binary_image)
# analyze regions
regions = regionprops_table(image_labeled, intensity_image=image_rescaled, properties = ('label','centroid', 'area', 'perimeter',
'equivalent_diameter', 'eccentricity', 'convex_area', 'mean_intensity'))

# turn into data frame for easy access
df = pd.DataFrame(regions)
df.shape

We implement a processing and filtering step to remove segmented objects that might not be bacteria colonies. We do a two step processing. The first step filters the regions based on their size and shape, selecting an area in between 60 - 500000 and an eccentricity (how rounded is an object) less than 0.8. T
The second filtering step involves removing colonies that are near or outside the border of the plate. This is done by fitting a circle to each region and checking if the circle lies within specified boundaries (200 to 1845 in both x and y directions). The code iterates through each region, calculates the circle's coordinates, and checks if all points of the circle are within the boundaries. If a region passes this check, its index is added to the `idx` list, and a unique label is assigned and stored in `label_ID`. Next, the code generates unique labels for each filtered region by combining a base name with the region's unique label. These labels are stored in the `name_label` list.

In [ ]:
# Filter in two steps
# filter first by size and sape
data_region = df.loc[(df['area']>60) & (df['area']<500000) & (df['eccentricity'] < 0.8)]
# filter second position on the plate
# calculate indexes to plot a circle and compare with the indexes in mask to remove colonies near or outside the border.
idx = []
label_ID = []
j = 1
th = np.arange(0,2*np.pi,np.pi/5)
for i in range(data_region.shape[0]):
    # fit a circle
    xunit = data_region.iloc[i][5]/2 * np.cos(th) + data_region.iloc[i][1]
    yunit = data_region.iloc[i][5]/2 * np.sin(th) + data_region.iloc[i][2]
    # find within the boundaries. Check ci variable
    y1 = np.array(yunit>200)
    y2 = np.array(yunit<1845)
    x1 = np.array(xunit>200)
    x2 = np.array(xunit<1845)
    xy = y1 & y2 & x1 & x2
    mean_xy = np.mean(np.multiply(xy, 1))
    if mean_xy == 1:
        idx.append(i)
        label_ID.append(j)
        j = j + 1
# filtered data frame
data_regions_filtered = data_region.iloc[idx]
data_regions_filtered.reset_index(drop=True, inplace=True)
data = data_regions_filtered[data_regions_filtered.columns[1:]]
name_label = []
for i in range(data.shape[0]):
    label_str = str(label_ID[i])
    name_label.append(str(image_id)+'_'+label_str)

temp_df = {'unique_label':name_label, 'label':label_ID}
name_df = pd.DataFrame(data=temp_df)
data_label = pd.concat([name_df, data], axis=1)

In [ ]:
data_label.shape

In [ ]:
data_label.head()

#### 4. Map the colonies in plate
We finalised the colonia identification by viualising labeled regions on the original image by drawing circles and labels, and then resizing the image for display.

In [ ]:
# make a copy of the original image
img = image_color.copy()
for i in range(data_label.shape[0]) :
    # draw circle
    cv2.circle(img,(round(data_label['centroid-1'][i]), round(data_label['centroid-0'][i])), round(data_label['equivalent_diameter'][i]/2), (0,255,0), 3)
    # draw text, label
    cv2.putText(img, str(data_label['label'][i]), (round(data_label['centroid-1'][i]+30), round(data_label['centroid-0'][i]+10)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (1, 1, 1), 4)

# new width and height to resize the image
scale = 50
width = int(img.shape[1] * scale/100)
height = int(img.shape[0] * scale/100)

# resizing the image
resized_img = cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

show_image(resized_img)


## Take home message
- Easy access to data using OMERO
- Thresholding segmentation can be use as an exploratory step for image segmentation